In [2]:
from folium import Map, Marker, Icon, Popup, IFrame, LayerControl
from folium.plugins import FeatureGroupSubGroup
from folium.map import FeatureGroup
import numpy as np
from collections import defaultdict
from branca.element import MacroElement
from jinja2 import Template

In [4]:
# Helper functions
# Convert rating (e.g. "3,8") to float (e.g. 3.8)
def parse_score(score):
    try:
        return float(score)
    except:
        return None

# Define colour for difficulty statistic
def difficulty_color(difficulty):
    if difficulty >= 9:
        return 'black'
    elif difficulty >= 7:
        return 'darkred'
    elif difficulty >= 5:
        return 'red'
    elif difficulty >= 3:
        return 'orange'
    else:
        return 'green'

# Create a difficulty rating: 
difficulty = (elevation_gain * gradient) / 100 + length * 5



In [81]:
# load JSON data
with open('passes/full_pass_data.json', 'r') as f:
    passes = json.load(f)

# compute difficulty score
for p in passes:
    length = p.get("length")
    elevation_gain = p.get("elevation_gain")
    gradient = p.get("gradient")

    if all(val is not None for val in [length, elevation_gain, gradient]):
        # Composite formula: effort + endurance
        raw_score = (elevation_gain * gradient) / 100 + length * 5
        p["composite_difficulty"] = round(raw_score, 1)
    else:
        p["composite_difficulty"] = None

# normalise to 0–10 scale
scores = [p["composite_difficulty"] for p in passes if p["composite_difficulty"] is not None]
min_score, max_score = min(scores), max(scores)

for p in passes:
    score = p.get("composite_difficulty")
    if score is not None:
        p["normalised_difficulty"] = round(10 * (score - min_score) / (max_score - min_score), 1)
    else:
        p["normalised_difficulty"] = None

# save updated data
with open("passes/passes_with_composite_difficulty.json", "w") as f:
    json.dump(passes, f, indent=2)

In [6]:
# load JSON data
with open('passes/passes_with_composite_difficulty.json', 'r') as f:
    data = json.load(f)

routes = data

In [8]:
# example route
routes[0]

{'name': 'Furkapass',
 'url_pass': 'https://www.quaeldich.de/paesse/furkapass/',
 'route_name': 'Ostrampe von Realp',
 'score': '4.5',
 'number_of_reviews': '31',
 'elevation': 2436,
 'deadend': 'not a dead end',
 'GPS': '46.5724,8.4150',
 'url_route': 'https://www.quaeldich.de/paesse/furkapass/profile/ostrampe-von-realp',
 'length': 13.0,
 'elevation_gain': 893,
 'gradient': 6.9,
 'url_gif': 'https://www.quaeldich.de/qdtp/anfahrten/55_88_gradient.gif',
 'composite_difficulty': 126.6,
 'normalised_difficulty': 3.7}

In [10]:
location_counts = defaultdict(int)

# create map and add layers 
m = Map(location=[46.8, 8.2], zoom_start=8)
base_group = FeatureGroup(name="All Routes").add_to(m) # all routes (not used for drawing here, just structural).

# score groups 
high_group = FeatureGroupSubGroup(base_group, "High Score (≥4)").add_to(m)
medium_group = FeatureGroupSubGroup(base_group, "Medium Score (2.5–4)").add_to(m)
low_group = FeatureGroupSubGroup(base_group, "Low Score (<2.5)").add_to(m)

for route in routes:
    try:
        lat, lon = map(float, route["GPS"].split(","))

        # Round lat/lon to 5 decimal places to avoid floating point quirks # don't want markers on top of each other)
        loc_key = (round(lat, 5), round(lon, 5))
        count = location_counts[loc_key]
        location_counts[loc_key] += 1

        # Apply a tiny offset in meters converted to lat/lon
        offset_meters = 1000  # adjust this value as needed
        angle = count * (2 * np.pi / 6)  # distribute in a circle (up to 6 markers before overlapping)
        offset_lat = (offset_meters / 111111) * np.cos(angle)  # approx meters per degree latitude
        offset_lon = (offset_meters / (111111 * np.cos(np.radians(lat)))) * np.sin(angle)

        lat += offset_lat
        lon += offset_lon
        
        score = parse_score(route.get("score"))
        difficulty = route.get("normalised_difficulty")
        length = route.get("length")
        gain = route.get("elevation_gain")
        gradient = route.get("gradient")
        deadend = route.get("deadend")

        # skip routes that don’t meet criteria
        if (
            score is None
            or length is None or gain is None or gradient is None or difficulty is None
        ):
            continue

        popup_html = f"""
            <b>{route.get('name', 'Unknown')}</b><br>
            Route: <i>{route.get('route_name', 'N/A')}</i><br>
            Max Height: {route.get('elevation', '?')} m<br>
            Elevation Gain: {gain} m<br>
            Length: {length} km<br>
            Gradient: {gradient}%<br>
            Score: {score}<br>
            Difficulty: {difficulty}<br>
            Dead End: {deadend}<br>
            <a href="{route.get('url_route', '#')}" target="_blank">Route Profile</a>
        """
        popup = Popup(IFrame(popup_html, width=300, height=200), max_width=300)

        # assign to the right group
        if score >= 4:
            score_group = high_group
        elif score >= 2.5:
            score_group = medium_group
        else:
            score_group = low_group
      
        marker = Marker(
            location=[lat, lon],
            tooltip=f"{route['name']} – Score: {score:.1f}",
            popup=Popup(IFrame(popup_html, width=300, height=200), max_width=300),
            icon=Icon(color=difficulty_color(difficulty), icon='bicycle', prefix='fa')
        )
        marker.add_to(score_group)

    except Exception as e:
        print(f"Skipping route due to error: {e}")

LayerControl(collapsed=False).add_to(m) # Adds a layer control widget to the map so users can toggle different layers.

# add legend manually (inject HTML)
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 160px; height: 190px;
    border:2px solid grey; z-index:9999; font-size:14px;
    background-color:white; padding: 10px;">
<b>Difficulty Legend</b><br>
<i class="fa fa-map-marker fa-2x" style="color:green"></i> Easy<br>
<i class="fa fa-map-marker fa-2x" style="color:orange"></i> Moderate<br>
<i class="fa fa-map-marker fa-2x" style="color:red"></i> Hard<br>
<i class="fa fa-map-marker fa-2x" style="color:darkred"></i> Very Hard<br>
<i class="fa fa-map-marker fa-2x" style="color:black"></i> Extreme
</div>
'''
# tools to insert raw HTML into a Folium map.
class Legend(MacroElement):
    def __init__(self, html):
        super().__init__()
        self._template = Template(f"""{{% macro html(this, kwargs) %}}{html}{{% endmacro %}}""")

m.get_root().add_child(Legend(legend_html))
#m.save("bike_routes_map.html")